In [1]:

import numpy as np
from cmaes._safe_cma import SafeCMA

In [5]:
dim = 5

def quadratic(x):
    coef = 1
    # coef = 1000 ** (np.arange(dim) / float(dim - 1))
    # coef = 1000 ** (np.arange(dim) / float(dim - 1))[::-1]
    
    return np.sum((x * coef) ** 2)

def safe_function1(x):
    return x[0] #+ np.random.randn() * 0.1

# def safe_function2(x):
#     return x[1]

safe_seeds = (np.random.rand(10, dim) * 2 - 1) * 5
safe_seeds[:,0] = - np.abs(safe_seeds[:,0])
# safe_seeds[:,1] = - np.abs(safe_seeds[:,1])
seeds_evals = np.array([ quadratic(x) for x in safe_seeds ])
seeds_safe_evals = np.stack([ [safe_function1(x)] for x in safe_seeds ])
# seeds_safe_evals = np.stack([ [safe_function1(x), safe_function2(x)] for x in safe_seeds ])
safety_threshold = np.array([0])

optimizer = SafeCMA(
    sigma=1., 
    safety_threshold=safety_threshold, 
    safe_seeds=safe_seeds,
    seeds_evals=seeds_evals,
    seeds_safe_evals=seeds_safe_evals,
)

In [6]:
print(f"#{optimizer._sigma} {optimizer.lip_penalty_coef}")

unsafe_eval_counts = 0
best_eval = np.inf

for generation in range(40 * dim):
    solutions = []
    for _ in range(optimizer.population_size):
        # Ask a parameter
        x = optimizer.ask()
        value = quadratic(x)
        safe_value = np.array([safe_function1(x)])
        # safe_value = np.array([safe_function1(x), safe_function2(x)])

        best_eval = np.min((best_eval, value))
        unsafe_eval_counts += (safe_value > safety_threshold)

        solutions.append((x, value, safe_value))
        # print(f"#{generation} {value} {safe_value} (x1={x[0]}, x2 = {x[1]})")

    # Tell evaluation values.
    optimizer.tell(solutions)

    print(f"#{generation * optimizer.population_size} ({best_eval} {unsafe_eval_counts})")
    
    if optimizer.should_stop():
        break

#0.006189470583944881 1
#0 (21.162701118926485 [0])
#8 (21.102686029358868 [0])
#16 (21.048062379887664 [0])
#24 (21.01651448995905 [0])
#32 (20.823372651925027 [0])
#40 (20.798236212609982 [0])
#48 (20.53132903779312 [0])
#56 (20.232824558838452 [0])
#64 (19.75414646069286 [0])
#72 (19.44549397806969 [0])
#80 (18.76400335900279 [0])
#88 (17.33277383019241 [0])
#96 (17.33277383019241 [0])
#104 (15.540438275464124 [0])
#112 (14.849474891772545 [0])
#120 (14.1458266067705 [0])
#128 (12.673855840263112 [0])
#136 (10.584398717170144 [0])
#144 (10.277575924357805 [0])
#152 (9.511847598328938 [0])
#160 (6.151707089898227 [0])
#168 (3.0056732397870993 [0])
#176 (2.3694049215524733 [0])
#184 (1.6720532508692632 [0])
#192 (0.9268341922912076 [0])
#200 (0.9268341922912076 [0])
#208 (0.9268341922912076 [0])
#216 (0.9268341922912076 [0])
#224 (0.9268341922912076 [0])
#232 (0.9268341922912076 [0])
#240 (0.9268341922912076 [0])
#248 (0.8095090274497616 [0])
#256 (0.7377883925669168 [0])
#264 (0.4679

In [4]:

"""
    example with single safety function
"""
def example1():
    # number of dimensions
    dim = 5

    # objective function
    def quadratic(x):
        coef = 1000 ** (np.arange(dim) / float(dim - 1)) 
        return np.sum((x * coef) ** 2)

    # safety function
    def safe_function(x):
        return x[0]

    # safe seeds
    safe_seeds_num = 10
    safe_seeds = (np.random.rand(safe_seeds_num, dim) * 2 - 1) * 5
    safe_seeds[:,0] = - np.abs(safe_seeds[:,0])

    # evaluation of safe seeds (with multiple safety functions)
    seeds_evals = np.array([ quadratic(x) for x in safe_seeds ])
    seeds_safe_evals = np.stack([ [safe_function(x)] for x in safe_seeds ])
    safety_threshold = np.array([0])

    # optimizer (safe CMA-ES)
    optimizer = SafeCMA(
        sigma=1., 
        safety_threshold=safety_threshold, 
        safe_seeds=safe_seeds,
        seeds_evals=seeds_evals,
        seeds_safe_evals=seeds_safe_evals,
    )

    unsafe_eval_counts = 0
    best_eval = np.inf

    for generation in range(400):
        solutions = []
        for _ in range(optimizer.population_size):
            # Ask a parameter
            x = optimizer.ask()
            value = quadratic(x)
            safe_value = np.array([safe_function(x)])

            # save best eval
            best_eval = np.min((best_eval, value))
            unsafe_eval_counts += (safe_value > safety_threshold)

            solutions.append((x, value, safe_value))

        # Tell evaluation values.
        optimizer.tell(solutions)

        print(f"#{generation} ({best_eval} {unsafe_eval_counts})")
        
        if optimizer.should_stop():
            break


"""
    example with multiple safety functions
"""
def example2():
    # number of dimensions
    dim = 5

    # objective function
    def quadratic(x):
        coef = 1000 ** (np.arange(dim) / float(dim - 1)) 
        return np.sum((x * coef) ** 2)

    # safety functions
    def safe_function1(x):
        return x[0]

    def safe_function2(x):
        return x[1]

    # safe seeds
    safe_seeds_num = 10
    safe_seeds = (np.random.rand(safe_seeds_num, dim) * 2 - 1) * 5
    safe_seeds[:,0] = - np.abs(safe_seeds[:,0])
    safe_seeds[:,1] = - np.abs(safe_seeds[:,1]) 

    # evaluation of safe seeds (with multiple safety functions)
    seeds_evals = np.array([ quadratic(x) for x in safe_seeds ])
    seeds_safe_evals = np.stack([ [safe_function1(x), safe_function2(x)] for x in safe_seeds ])
    safety_threshold = np.array([0,0])

    # optimizer (safe CMA-ES)
    optimizer = SafeCMA(
        sigma=1., 
        safety_threshold=safety_threshold, 
        safe_seeds=safe_seeds,
        seeds_evals=seeds_evals,
        seeds_safe_evals=seeds_safe_evals,
    )

    unsafe_eval_counts = 0
    best_eval = np.inf

    for generation in range(400):
        solutions = []
        for _ in range(optimizer.population_size):
            # Ask a parameter
            x = optimizer.ask()
            value = quadratic(x)
            safe_value = np.array([safe_function1(x), safe_function2(x)])

            # save best eval
            best_eval = np.min((best_eval, value))
            unsafe_eval_counts += (safe_value > safety_threshold)

            solutions.append((x, value, safe_value))

        # Tell evaluation values.
        optimizer.tell(solutions)

        print(f"#{generation} ({best_eval} {unsafe_eval_counts})")
        
        if optimizer.should_stop():
            break

In [5]:
# example1()

In [6]:
# example2()